In [1]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
import os
import re
from pprint import pprint
import argparse
import ast
from pathlib import Path
import pathlib
import re
import pprint 
import argparse
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import xmltojson
from pprint import pprint
import json 
p = Path('.')
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

def json_string_dict_values(json_string):
    collect_dict_values= {}
    for item in json_string.split('\n'):
        ii= item.split('=')
        if 'year' in [i.strip() for i in ii ]:
            collect_dict_values['year']=re.sub('\W+','-', item.split('=')[1])
        if 'title' in [i.strip() for i in ii ]:
            collect_dict_values['title']=re.sub('\W+','-', item.split('=')[1])
        if 'author' in [i.strip() for i in ii ]:
            collect_dict_values['author']=re.sub('\W+','-', item.split('=')[1])
            # print(re.sub('\W+','-', item.split('=')[1]))
        if 'journal' in [i.strip() for i in ii ]:
            collect_dict_values['journal']=re.sub('\W+','-', item.split('=')[1])
    return collect_dict_values


In [2]:
reference_file='/Users/krishnaneupane/Documents/personal/academic/financialanomalies/references.bib'
intro_qmd_file='/Users/krishnaneupane/Documents/personal/academic/financialanomalies/intro.qmd'

## manual user input
url= f'https://scholar.google.com/scholar?cites=17004099255623082404&as_sdt=5,47&sciodt=0,47&hl=en' 

# user manual inputcd 
json_string='''@article{lo2006trading,
  title={Trading volume: Implications of an intertemporal capital asset pricing model},
  author={Lo, Andrew W and Wang, Jiang},
  journal={The Journal of Finance},
  volume={61},
  number={6},
  pages={2805--2840},
  year={2006},
  publisher={Wiley Online Library}
}'''

collect_dict_values= json_string_dict_values(json_string)
print(''.join(('article-',collect_dict_values['year'],collect_dict_values['author'],
               collect_dict_values['title'], collect_dict_values['journal'][:-1],'.pdf',)))
citation_key= [item.split(',') for item in json_string.split('\n')][0][0].split('{')[1]
print('CITATION KEY: ', citation_key)
os.getcwd()

article--2006--Lo-Andrew-W-and-Wang-Jiang--Trading-volume-Implications-of-an-intertemporal-capital-asset-pricing-model--The-Journal-of-Finance.pdf
CITATION KEY:  lo2006trading


'/Users/krishnaneupane/Documents/personal/academic/academic_graphs'

In [3]:
os.chdir('/Users/krishnaneupane/Documents/personal/academic/academic_graphs')

with open (reference_file, 'a') as reffile:
    ref_file=reffile.write(f'\n{json_string}')
    reffile.close()

citation_key =f'{citation_key}' # user manual input

with open (intro_qmd_file, 'a') as introQmdFile: ## insert citation key at the end of the main page
    ref_file=introQmdFile.write(f'\n@{citation_key}')
    introQmdFile.close()

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
}
page_contents=requests.get(url,headers=headers)
citation_dict={}
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=cited_by_gscholar(BeautifulSoup(page_contents.text,'lxml'))
df = pd.DataFrame.from_dict(citation_dict)
df.to_csv('academic_citation_network.csv', mode='a')

!clear
!export citation_key=($citation_key)
!git add .
!git status
!git commit -m "$citation_key"
!git push
!clear 

# docker run --rm --init --ulimit core=0 -p 8070:8070 lfoppiano/grobid:0.8.0
citation_dict


In [4]:
import os
os.chdir("/Users/krishnaneupane/Documents/personal/academic/financialanomalies")
print(os.getcwd())
!sh commitRender.sh $citation_key


/Users/krishnaneupane/Documents/personal/academic/financialanomalies
[main ad6a45c] fama1992cross
 7 files changed, 336 insertions(+), 37 deletions(-)
Enumerating objects: 17, done.
Counting objects: 100% (17/17), done.
Delta compression using up to 16 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 41.87 KiB | 5.98 MiB/s, done.
Total 9 (delta 8), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (8/8), completed with 8 local objects.
To https://github.com/krishpn/financialanomalies.git
   444e831..ad6a45c  main -> main
[1/4] index.qmd
[2/4] intro.qmd
[3/4] summary.qmd
[4/4] references.qmd

pandoc 
  to: latex
  output-file: index.tex
  standalone: true
  toc: true
  number-sections: true
  top-level-division: chapter
  pdf-engine: xelatex
  variables:
    graphics: true
    tables: true
  default-image-extension: pdf
  
metadata
  crossref:
    chapters: true
  documentclass: scrreprt
  papersize: letter
  classoption:
    - DIV=11
    - numb

In [5]:
with open('/Users/krishnaneupane/Downloads/testGorbid/ssrn-3604626.grobid.tei.xml', 'r') as f:
    my_xml = f.read()
dd= json.loads(xmltojson.parse(my_xml))


main_pub_root= dd.get('TEI').get('teiHeader').get('fileDesc').get('sourceDesc').get('biblStruct')

main_pub_date= main_pub_root.get('monogr').get('imprint').get('date').get("#text")


main_pub_title = main_pub_root.get('analytic').get('title').get("#text")


for item in main_pub_root.get('analytic').get('author'):
    fname= item.get('persName').get('forename')

    if  isinstance(fname, list):
        flname=[i['#text'] for i in fname]
        lname= item['persName'].get('surname')
        flname= flname + [lname]
        flname= ' '.join(flname)
        print(flname,lname,main_pub_title, main_pub_date)
    else:
        fname= item.get('persName').get('forename').get('#text')
        lname= item['persName'].get('surname')
        flname= fname +' '+ lname
        print(flname, lname,main_pub_title, main_pub_date)

for item in dd['TEI']['text']['back']['div']:
    if 'listBibl' in list(item.keys()):
        for sitem in item['listBibl']['biblStruct']:
            journal = sitem.get('monogr').get('title').get('#text')
            try: 
                title= sitem['analytic']['title']['#text']
    
                for stitle in sitem['analytic']['author']:
                    fname= stitle.get('persName').get('forename')
                    if  isinstance(fname, list):
                        flname=[i['#text'] for i in fname]
                        lname= stitle['persName'].get('surname')
                        flname= flname + [lname]
                        flname= ' '.join(flname)
                        year_pub = sitem.get('monogr').get('imprint').get('date').get('#text')
                        print(year_pub, title,flname, journal)
                    else:
                        fname= stitle.get('persName').get('forename').get('#text')
                        lname= stitle['persName'].get('surname')
                        flname= fname +' '+ lname
                        year_pub = sitem.get('monogr').get('imprint').get('date').get('#text')
                        print(year_pub, title, flname, journal)
            except Exception:
                pass

Andrew Y Chen Chen Open Source Cross-Sectional Asset Pricing May 2021
Tom Zimmermann Zimmermann Open Source Cross-Sectional Asset Pricing May 2021
1998 Abnormal returns to a fundamental analysis strategy Jeffery S Abarbanell Accounting Review
1998 Abnormal returns to a fundamental analysis strategy Brian J Bushee Accounting Review
2005 Asset pricing with liquidity risk Viral V Acharya Journal of financial Economics
2005 Asset pricing with liquidity risk Lasse Heje Pedersen Journal of financial Economics
2014 Financial intermediaries and the cross-section of asset returns Tobias Adrian The Journal of Finance
2014 Financial intermediaries and the cross-section of asset returns Erkko Etula The Journal of Finance
2014 Financial intermediaries and the cross-section of asset returns Tyler Muir The Journal of Finance
2003 Arbitrage risk and the bookto-market anomaly Ashiq Ali Journal of Financial Economics
2003 Arbitrage risk and the bookto-market anomaly Lee-Seok Hwang Journal of Financial E